In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [ ]:
# 선수 ID 지정 (예: 73330은 특정 선수의 고유 번호)
player_id = "11310"
url = f"https://statiz.sporki.com/player/?m=year&p_no={player_id}"
# url ='https://statiz.sporki.com/stats/?m=team&m2=pitching'


In [ ]:
def crawl_statiz_pitcher_data(url):
    """
    스탯티즈에서 선발투수 등판 기록을 크롤링하는 함수
    """
    try:
        # HTTP 요청 헤더 설정 (봇 차단 방지)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # 웹페이지 요청
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 테이블 찾기 (일반적으로 통계 데이터는 table 태그 안에 있음)
        table = soup.find('table')
        if not table:
            print("테이블을 찾을 수 없습니다.")
            return None
        
        # 헤더 추출 (th 태그 또는 첫 번째 tr의 td 태그)
        headers = []
        header_row = table.find('tr')
        if header_row:
            header_cells = header_row.find_all(['th', 'td'])
            headers = [cell.get_text(strip=True) for cell in header_cells]
        
        # 데이터 행 추출
        data_rows = []
        rows = table.find_all('tr')[1:]  # 첫 번째 행(헤더) 제외
        
        for row in rows:
            cells = row.find_all('td')
            if cells:  # td 태그가 있는 행만 처리
                row_data = []
                for cell in cells:
                    # 셀 내용 추출 (줄바꿈 제거, 공백 정리)
                    text = cell.get_text(separator=' ', strip=True)
                    # 추가 줄바꿈 문자 제거
                    text = text.replace('\n', ' ').replace('\r', '').strip()
                    # 연속된 공백을 하나로 통일
                    text = ' '.join(text.split())
                    row_data.append(text)
                
                if row_data:  # 빈 행이 아닌 경우만 추가
                    data_rows.append(row_data)
        
        return headers, data_rows
        
    except requests.RequestException as e:
        print(f"웹페이지 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")
        return None

In [ ]:
picherdata=crawl_statiz_pitcher_data(url)
picherdata

In [ ]:
picherdata_DF=pd.DataFrame(picherdata[1])
picherdata_DF = picherdata_DF.set_axis(picherdata[0], axis=1)
picherdata_DF


In [ ]:
picherdata_DF.iloc[-1]